# Zero to Cortex  
## *Intermediate*
This portion of the lab will go through some of the basic principles of creating a Retrieval Augmented Generation application and using LLM's for Data Engineering problems.  
https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions

![Alt text](https://venturebeat.com/wp-content/uploads/2024/04/a-robot-playing-with-a-snowflake-in-arctic-cinemat-4OYW23nATBm50aD_slLk8w-xaTFE1EbSLmDWJXvWCxXrA.jpeg?fit=750%2C422&strip=all "a title")

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
from snowflake.snowpark import functions as F
from snowflake.cortex import Complete

## How to build the underpinnings of a Retrieval Augmented Generation (RAG) application  
In this section, you will build the infrastructure for a RAG application that helps users find wines by interacting with an AI sommelier. The steps to build the infrastructure are as follows:  
1. If your unstructured data requires chunking, do this first. Chunking will split apart the unstructured data into pieces that are smaller (*usually 300-1000 words, but experiment with this!*) that overlap each other to some degree. Each chunk is grouped by its parent data. While this can be done in SQL, Python has great support for doing things like this.
    * In most cases, LangChain's [RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html#langchain_text_splitters.character.RecursiveCharacterTextSplitter) is used.
2. Once you have prepared your unstructured data, now you can create embeddings. In Snowflake, it is really easy to create embeddings of your unstructured data. You'll want to create a new column in your table or just create a new table that is used for your RAG application. Here is an example of how you would do this:
    * ```  
        select
            text_field,
            text_chunk,
            snowflake.cortex.embed_text_768(
                'snowflake-arctic-embed-m',
                text_chunk
            ) as chunk_embedding
        from
            my_table
        ```
3. Now you can run similarity queries against your data to get the most relevant chunks of data to feed to your LLM as context. Here is an example of how to do get the top five most relevant pieces of information:  
    * ```
        select
            text_chunk
        from
            (
            select
                vector_cosine_similarity(
                    chunk_embedding,
                    snowflake.cortex.embed_text_768(
                        'snowflake-arctic-embed-m',
                        '<your text/question will go here>'
                    )
                ) as similarity
            from
                my_table
            order by
                similarity
            limit 5
            )
        ```
4. The result from the above query will give you the most relevant chunks of data to provide to your LLM as context. We would combine those chunks of data using Python to feed over to the LLM. Here is how you would combine that result set and provide it to the LLM:  
    * ```
            # model_name will be the model you select
        # This can be Snowflake Arctic or any of the other models we support in Cortex!
        model_name = 'snowflake-arctic'
        question = """<Question from the chat interface>"""
        chunks = session.sql(f"""
            select
                text_chunk
            from
                (
                select
                    vector_cosine_similarity(
                        chunk_embedding,
                        snowflake.cortex.embed_text_768(
                            'snowflake-arctic-embed-m',
                            '{question}'
                        )
                    ) as similarity
                from
                    my_table
                )
            order by
                similarity
            limit 5""")
        info = '. | '.join([x[0] for x in chunks.select("*").collect()]).replace("'", "")
        prompt = f"""
                    <YOUR PROMPT>
                    Answer the questions based on the context provided between the <context> and </context> tags. The
                    question will be found between the <question> and </question> tags.
                    <context>
                    '{info}'
                    </context>
                    <question>
                    '{info}'
                    </question>
                    Answer: """
        query = """
                select
                    snowflake.cortex.complete(
                    # model name goes here
                        ?, 
                    # prompt goes here
                        ?
                    ) as response
                """
        complete = session.sql(query, params=[model_name, prompt])
        complete.collect()[0][0]
        ```
**That's it!** You have successfully created the underpinnings of a RAG application. Your LLM will have pointed context to your questions. You can optionally add some more complexity here with LLM hyperparameters or by also providing the chat history for further context. Additionally, when we are dealing with millions of rows of data, we should opt to use the Snowflake Cortex Search API. This is a brand new offering (currently in Private Preview as of 5/3/24).  
``` Cortex Search enables low-latency, high-quality search over your Snowflake data.
Cortex Search powers a broad array of search experiences for Snowflake users including 
Retrieval Augmented Generation (RAG) applications leveraging Large Language Models (LLMs).

Cortex Search gets you up and running with a vector and keyword-based search engine on your
text data in minutes, without having to worry about embedding, infrastructure maintenance, 
search quality parameter tuning, or ongoing index refreshes. This means you can spend less 
time on infrastructure and search quality tuning, and more time developing high-quality 
chat and search experiences using your data.
```  
## Let's run some sample queries!

In [ ]:
model_name = st.radio(
    label = "Choose your model",
    options = [
        "snowflake-arctic",
        "mistral-large",
        "mixtral-8x7b",
        "mistral-7b",
        "reka-flash",
        "reka-core",
        "llama2-70b-chat",
        "llama3-70b",
        "llama3-8b",
        "gemma-7b"
    ]
)

In [ ]:
# Enter your question here!
question = """I need a list of the top three white wines to pair well with seafood. 
                Please provide the wine, some information about the wine, and the price. Also, add up the 
                total cost of the list."""

In [ ]:
# Get the relevant data
chunks = session.sql(f"""
  select
      full_description
  from
      (
        select
            full_description,
            vector_cosine_similarity(
                information_embeds,
                snowflake.cortex.embed_text_768(
                  'snowflake-arctic-embed-m',
                  '{question}'
                )
            ) as similarity
        from
            wine_reviews
        )
  order by
      similarity
  limit 10""")
chunks.show()

In [ ]:
# Give the context to the LLM and get your question answered!
info = '. | '.join([x[0] for x in chunks.select("*").collect()]).replace("'", "")
prompt = f"""
            You are a world class sommelier. Our guests seek knowledge and guidance. Do not hallucinate, the guests don't like that.
            If you don't know what to recommend just say so. Please answer the questions based on the provided 
            context found between the tags <context> and </context>. The guest's question will be between the
            <question> and </question> tags. Please present the wine nicely. Explain where it is from, the variety of wine it is, and the price.
            Answer the questions based on the context provided between the <context> and </context> tags. The
            question will be found between the <question> and </question> tags.
            <context>
            '{info}'
            </context>
            <question>
            '{question}'
            </question>
            Answer: """
query = """
      select
          snowflake.cortex.complete(
              ?, 
              ?
          ) as response
      """
complete = session.sql(query, params=[model_name, prompt])
with st.chat_message(name="Assistant"):
    st.write(complete.collect()[0][0])

## Another interesting use case - *data cleaning*  
In this next example, you will see that one would have to write some Regex to extract information from the `variable` column. What if we could do this with LLMs instead of writing complex code? 

In [ ]:
select * from sec_filings limit 5

In [ ]:
-- Create a new column called CLASSIFICATION from the VARIABLE column
-- This column will give you a clean document type classification
select
    variable,
    snowflake.cortex.complete(
        'snowflake-arctic',
        CONCAT('Based on the value between the <variable> and </variable> tags, please classify
            the data in ONLY one of these three categories: 10K, 10Q, 8K. If you cannot classify
            the data based on the information, impute NULL. Do not provide an explanation. Only provide 
            your answer of 10K, 10Q, 8K, or NULL.
            <variable>', variable, '</variable>'
        )) as classification
from
    sec_filings
limit 5